# Predicting links in Social Networks

In [76]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pickle

## Load data

### Like relation

In [77]:
like_1 = pd.read_csv("data/engineered_features/Like_Features_day_1.csv")
like_2 = pd.read_csv("data/engineered_features/Like_Features_day_2.csv")
like_3 = pd.read_csv("data/engineered_features/Like_Features_day_3.csv")
like_4 = pd.read_csv("data/engineered_features/Like_Features_day_4.csv")
like_5 = pd.read_csv("data/engineered_features/Like_Features_day_5.csv")

In [78]:
common_1 = pd.read_csv("data/engineered_features/common_interest_day_1.csv")
common_2 = pd.read_csv("data/engineered_features/common_interest_day_2.csv")
common_3 = pd.read_csv("data/engineered_features/common_interest_day_3.csv")
common_4 = pd.read_csv("data/engineered_features/common_interest_day_4.csv")
common_1_3 = pd.read_csv("data/engineered_features/common_interest_combined_day_1_3.csv")

### Group Relation

In [79]:
with open("data/engineered_features/G_features_postings_days_1-3.pkl", "rb") as input_file:
    G_1_3 = pickle.load(input_file)

### Posting TFIDF + graph features

In [80]:
with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_days_1to3.pkl", "rb") as input_file:
    tfidf_pos_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_day_4_test.pkl",
          "rb") as input_file:
    tfidf_pos_4_test = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_days_1to3.pkl", "rb") as input_file:
    tfidf_neg_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_day_4_test.pkl",
          "rb") as input_file:
    tfidf_neg_4_test = pickle.load(input_file)

## Process data for ML

### Downsample data

Experiment paramters:

In [81]:
TRAIN_SET_SPLIT_SIZE = 0.8
TEST_SET_DOWN_FACTOR = 0.3
TRAIN_SET_DOWN_FACTOR = 0.7
TRAIN_SET_NEG_OVERBALANCE = 2
PREDICTION_GOAL = "DAY_4"
#PREDICTION_GOAL = "SAME_DAY"

#Exludes long-running classifiers
QUICK_RUN = True

Because of the huge number of nodes execution of basic data handling takes long on commodity hardware. Therefore the data sets need to be reduced in size while keeping the characteristics of the orignal data set

#### For the common interest relationship

In [82]:
if PREDICTION_GOAL == "SAME_DAY":
    com = common_1_3
    com["label"] = [1 if c > 0 else 0 for c in common_1_3["weight"]]
elif PREDICTION_GOAL == "DAY_4":
    com_4 = common_4[["source","target","weight"]]
    merged_com = pd.merge(common_1_3, com_4, on=["source","target"], how="left")
    merged_com.rename(columns= {"weight_y": "day_4_weight"}, inplace=True)
    merged_com['day_4_weight'] = merged_com['day_4_weight'].fillna(0)
    com = merged_com
    com["label"] = [1 if c > 0 else 0 for c in merged_com["day_4_weight"]]

In [83]:
com_train, com_test = train_test_split(com, train_size=TRAIN_SET_SPLIT_SIZE)

com_test_downsample = resample(
    com_test,
    n_samples=int(com_test.shape[0] * TEST_SET_DOWN_FACTOR)
)

com_train_neg = com_train[com_train.label == 0]
com_train_pos = com_train[com_train.label == 1]

com_train_neg_downsample = resample(com_train_neg,
                              n_samples=int(com_train_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                              )

com_train_resampled = resample(
    pd.concat([com_train_pos, com_train_neg_downsample]),
    n_samples=int( (com_train_pos.shape[0] + com_train_neg_downsample.shape[0]) * TRAIN_SET_DOWN_FACTOR)
)

In [84]:
com_train_resampled.shape[0]

66389

The resampled data structure still contains over 200k links (not considering train set downscaling), but now the label imbalance is corrected.

#### For the like relationship

In [85]:
if PREDICTION_GOAL == "SAME_DAY":
    like_all = pd.concat([like_1, like_2, like_3, like_4, like_5])
    like_all["label"] = [1 if c > 0 else 0 for c in like_all["current_weight"]]
elif PREDICTION_GOAL == "DAY_4":
    like_1_3 = pd.concat([like_1, like_2, like_3])

    like_4 = like_4[["source","target","current_weight"]]
    merged_like = pd.merge(like_1_3, like_4, on=["source","target"], how="left")
    merged_like.rename(columns= {"current_weight_y": "day_4_weight"}, inplace=True)
    merged_like['day_4_weight'] = merged_like['day_4_weight'].fillna(0)
    like_all = merged_like
    like_all["label"] = [1 if c > 0 else 0 for c in like_all["day_4_weight"]]

In [86]:
like_train, like_test = train_test_split(like_all, train_size=TRAIN_SET_SPLIT_SIZE)


like_test_downsampled = resample(
    like_test,
    n_samples=int(like_test.shape[0] * TEST_SET_DOWN_FACTOR)
)

like_train_neg = like_train[like_train.label == 0]
like_train_pos = like_train[like_train.label == 1]

like_train_neg_downsampled = resample(like_train_neg,
                                n_samples=int(like_train_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                                )
like_train_resampled = resample(
    pd.concat([like_train_pos, like_train_neg_downsampled]),
    n_samples=int( (like_train_pos.shape[0] + like_train_neg_downsampled.shape[0]) * TRAIN_SET_DOWN_FACTOR)
)

In [87]:
like_train_resampled.shape[0]

32753

The resampled data structure still contains over 400k (not considering train set downscaling) links, but now the label imbalance is corrected.

#### For group relation using TFIDF


In [88]:
if PREDICTION_GOAL == "SAME_DAY":
    tfidf_all_train = pd.concat([tfidf_neg_1_3, tfidf_pos_1_3])
    tfidf_all_test = pd.concat([tfidf_neg_4_test, tfidf_pos_4_test])
    tfidf_all = pd.concat([tfidf_all_train, tfidf_all_test])
elif PREDICTION_GOAL == "DAY_4":
    tfidf_1_3 = pd.concat([tfidf_neg_1_3, tfidf_pos_1_3])
    tfidf_4 = pd.concat([tfidf_neg_4_test, tfidf_pos_4_test])

    tfidf_4 = tfidf_4[["Source_User","Target_User","label"]]

    merged_tfidf = pd.merge(tfidf_1_3, tfidf_4, on=["Source_User","Target_User"], how="left")
    merged_tfidf.rename(columns= {"label_y": "label"}, inplace=True)
    merged_tfidf['label'] = merged_tfidf['label'].fillna(0)
    tfidf_all = merged_tfidf

In [89]:
tfidf_train, tfidf_test = train_test_split(tfidf_all, train_size=TRAIN_SET_SPLIT_SIZE)


tfidf_test_downsampled = resample(
    tfidf_test,
    n_samples=int(tfidf_test.shape[0] * TEST_SET_DOWN_FACTOR)
)

tfidf_train_neg = tfidf_train[tfidf_train.label == 0]
tfidf_train_pos = tfidf_train[tfidf_train.label == 1]

tfidf_train_neg_downsampled = resample(tfidf_train_neg,
                                n_samples=int(tfidf_train_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                                )
tfidf_train_resampled = resample(
    pd.concat([tfidf_train_pos, tfidf_train_neg_downsampled]),
    n_samples=int( (tfidf_train_pos.shape[0] + tfidf_train_neg_downsampled.shape[0]) * TRAIN_SET_DOWN_FACTOR)
)

In [90]:
tfidf_train_resampled.shape[0]

5760

The resampled data structure still contains over 90k (not considering train set downscaling) links, but now the label imbalance is corrected.

## Define columns for training

In [91]:
target = "label"
graph_features = [
    "jaccard_coef",
    "adamic_adar_index",
    "preferential_attachment_index",
    "clustering_coefficient_score"
]
larger_graph_features = [
    "jaccard_coef",
    "adamic_adar_index",
    "preferential_attachment_index",
    "clustering_coefficient_score_Source_User",
    "clustering_coefficient_score_Target_User",
    "pagerank_Source_User",
    "pagerank_Target_User"
]
tfidf_features = [ f'{i}_TFIDF' for i in range(0,500)]


X_train_com = com_train_resampled[graph_features]
y_train_com = com_train_resampled[target]

X_train_like = like_train_resampled[graph_features]
y_train_like = like_train_resampled[target]

X_train_tfidf = tfidf_train_resampled[larger_graph_features + tfidf_features]
y_train_tfidf = tfidf_train_resampled[target]

In [92]:
rfc_com = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_com = SVC()
knn_com = KNeighborsClassifier(n_neighbors=10)

rfc_like = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_like = SVC()
knn_like = KNeighborsClassifier(n_neighbors=10)

rfc_tfidf = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_tfidf = SVC()
knn_tfidf = KNeighborsClassifier(n_neighbors=10)

## Define evaluation function

In [93]:
def evaluate_model(predictions, actual):
    return pd.DataFrame({
        "Measure": ["Accuracy", "Precision", "Recall", "F1"],
        "Score": [accuracy_score(actual, predictions),
                  precision_score(actual, predictions),
                  recall_score(actual, predictions),
                  f1_score(actual, predictions)]
    }).transpose()

## Fit classifiers

### RF

In [94]:
rfc_com.fit(X_train_com, y_train_com)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

In [95]:
rfc_like.fit(X_train_like, y_train_like)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

In [96]:
rfc_tfidf.fit(X_train_tfidf, y_train_tfidf)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

### SVM

In [97]:
if not QUICK_RUN:
    svm_com.fit(X_train_com, y_train_com)

In [98]:
if not QUICK_RUN:
    svm_like.fit(X_train_like, y_train_like)

In [99]:
if not QUICK_RUN:
    svm_tfidf.fit(X_train_tfidf, y_train_tfidf)

### KNN

In [100]:
knn_com.fit(X_train_com, y_train_com)

KNeighborsClassifier(n_neighbors=10)

In [101]:
knn_like.fit(X_train_like, y_train_like)

KNeighborsClassifier(n_neighbors=10)

In [102]:
knn_tfidf.fit(X_train_tfidf, y_train_tfidf)

KNeighborsClassifier(n_neighbors=10)

## Evaluate predictions

In [103]:
X_test_com = com_test_downsample[graph_features]
y_test_com = com_test_downsample[target]

In [104]:
X_test_like = like_test_downsampled[graph_features]
y_test_like = like_test_downsampled[target]

In [105]:
X_test_tfidf = tfidf_test_downsampled[larger_graph_features + tfidf_features]
y_test_tfidf = tfidf_test_downsampled[target]

### RF evaluation

In [121]:
predictions = rfc_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.90932,0.016174,0.769031,0.031681


In [107]:
predictions = rfc_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.832627,0.003084,0.345865,0.006114


In [108]:
predictions = rfc_tfidf.predict(X_test_tfidf)
evaluate_model(predictions, y_test_tfidf)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.969081,0.389635,0.894273,0.542781


### SVM evaluation

In [109]:
if not QUICK_RUN:
    predictions = svm_com.predict(X_test_com)
    evaluate_model(predictions, y_test_com)

In [110]:
if not QUICK_RUN:
    predictions = svm_like.predict(X_test_like)
    evaluate_model(predictions, y_test_like)

In [111]:
if not QUICK_RUN:
    predictions = svm_tfidf.predict(X_test_tfidf)
    evaluate_model(predictions, y_test_tfidf)

### KNN evaluation

In [112]:
predictions = knn_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.90787,0.014844,0.715398,0.029085


In [113]:
predictions = knn_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.852822,0.002716,0.267335,0.005378


In [114]:
predictions = knn_tfidf.predict(X_test_tfidf)
evaluate_model(predictions, y_test_tfidf)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.936353,0.221704,0.837004,0.350554


## Check Train set imbalance

### COM

In [115]:
(y_train_com == 0).sum()/y_train_com.shape[0]

0.6667369594360512

In [116]:
(y_test_com == 0).sum()/y_test_com.shape[0]

0.9980710587901567

### LIKE

In [117]:
(y_train_like == 0).sum()/y_train_like.shape[0]

0.6634201447195677

In [118]:
(y_test_like == 0).sum()/y_test_like.shape[0]

0.9985116457194583

### TFID

In [119]:
(y_train_tfidf == 0).sum()/y_train_tfidf.shape[0]

0.671875

In [120]:
(y_test_tfidf == 0).sum()/y_test_tfidf.shape[0]

0.9794774432691439

# TODO

- Combine different features
- Visualize predicted networks?
- Grid search for params?
- Cross validation?
- Explainable?


See this [repo](https://github.com/neo4j-examples/link-prediction) for reference